In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset=pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
dataset=pd.DataFrame(dataset)
dataset.head()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
DATA_DIR="../input/cassava-leaf-disease-classification"
TEST_DATA="../input/cassava-leaf-disease-classification/test_images"
TRAIN_DATA="../input/cassava-leaf-disease-classification/train_images"
csv_file="../input/cassava-leaf-disease-classification/train.csv"

In [ ]:
labels=dataset.label.unique()


In [ ]:
def one_hot_encoding(data):
    output=data.label
    data[str(output)]=1
    
    return data

In [ ]:
for i in labels:
    dataset[str(i)]=0

In [ ]:
dataset=dataset.apply(one_hot_encoding,axis=1)
dataset.head()

In [ ]:
Target_Size=(224,224)
from tensorflow.keras.preprocessing import image
def show_image(image_path):
    imag=image.load_img(image_path,target_size=Target_Size)
    x=image.img_to_array(imag)
    x=x/255
    plt.imshow(x)

In [ ]:
show_image("../input/cassava-leaf-disease-classification/train_images/1000015157.jpg")

In [ ]:
Y=list(dataset.columns[2:])
Y

In [ ]:
Train_Data=ImageDataGenerator(rescale=1/255.,
                             validation_split=0.2,
                            rotation_range=90,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              horizontal_flip=True,
                              vertical_flip=True,
                             )
Train=Train_Data.flow_from_dataframe(
        dataframe=dataset,
        directory=TRAIN_DATA,
        x_col="image_id",
        y_col=Y,
        shuffle=True,
        batch_size=32,
        subset="training",
        class_mode="raw",
        target_size=Target_Size,
        seed=42
    )

Validation=Train_Data.flow_from_dataframe(
        dataframe=dataset,
        directory=TRAIN_DATA,
        x_col="image_id",
        y_col=Y,
        shuffle=True,
        batch_size=32,
        subset="validation",
        class_mode="raw",
        target_size=Target_Size,
        seed=42
    )

In [ ]:
# !mkdir ~/.keras
# !mkdir ~/.keras/models
# !cp ../input/keras-pretrained-models/*notop* ~/.keras/models/
# !cp ../input/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/

In [ ]:
D=tf.keras.applications.Xception(
    include_top=False, weights='../input/keras-pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5', input_shape=(224,224,3))

inp = keras.layers.Input([224, 224, 3])
x = tf.keras.applications.xception.preprocess_input(inp)
x=D(x)

In [ ]:
from tensorflow.keras import Model
x=GlobalAveragePooling2D()(x)
x=Dropout(0.4)(x)
out=Dense(5,activation="softmax")(x)
model=Model(inp,out)
model.summary()
model.compile(optimizer=Adam(lr=(0.001)),loss="categorical_crossentropy", metrics='accuracy')

In [ ]:
ee=EarlyStopping(monitor='val_loss', patience=10, verbose=1)
history=model.fit(
Train,
epochs=5,
steps_per_epoch=17118//5,
validation_data=Validation,
validation_steps=4279//5,
callbacks=[EarlyStopping(monitor = 'val_accuracy', min_delta = 1e-4, patience = 20, mode = 'max', 
                    restore_best_weights = True, verbose = 0),ModelCheckpoint('model.h5',monitor = 'val_accuracy',
                      verbose = 0, save_best_only = True, mode = 'max')]
)


In [ ]:
show_image("../input/cassava-leaf-disease-classification/test_images/2216849948.jpg")

In [ ]:
from PIL import Image
def predict_(path):
    img=Image.open(path)
    img=img.resize((224,224))
    img=np.expand_dims(img,axis=0)
    y_pred=model.predict(img)
    return y_pred

In [ ]:
y_pred=predict_("../input/cassava-leaf-disease-classification/test_images/2216849948.jpg")

In [ ]:
print(np.argmax(y_pred))

In [ ]:
counter=[]
for k,i in enumerate(os.listdir(TEST_DATA)):
    df={'image_id':i,'label':labels[np.argmax(predict_(TEST_DATA+'/'+i))]}
    counter.append(k)

In [ ]:
df=pd.DataFrame(df,index=counter)
df

In [ ]:
df.to_csv('submission.csv',index=False)
